# TAPE

In [2]:
import torch
from tape import ProteinBertModel, TAPETokenizer

/home/vicente/anaconda3/envs/torch11/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = ProteinBertModel.from_pretrained('bert-base')
tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model
#print(model)

In [5]:
sequence = 'GCTVEDRCLIGMGAILLNGCVIGSGSLVAAGALITQ'
encoding = tokenizer.encode(sequence)
print(encoding)
print(encoding.shape)
token_ids = torch.tensor([tokenizer.encode(sequence)])

[ 2 11  7 23 25  9  8 21  7 15 13 11 16 11  5 13 15 15 17 11  7 25 13 11
 22 11 22 15 25  5  5 11  5 15 13 23 20  3]
(38,)


/tmp/ipykernel_11350/2687810853.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/utils/tensor_new.cpp:210.)
  token_ids = torch.tensor([tokenizer.encode(sequence)])


In [6]:
output = model(token_ids)
sequence_output = output[0]
pooled_output = output[1]
print(sequence_output.shape)
print(sequence_output)

torch.Size([1, 38, 768])
tensor([[[ 0.9592,  0.7883, -0.2636,  ...,  0.2139,  0.6012, -1.0673],
         [ 0.2163, -0.5777, -1.0532,  ..., -1.1601, -0.4544, -0.9216],
         [ 2.7015,  0.1971, -0.3688,  ...,  0.7066, -1.1335, -0.6094],
         ...,
         [-0.5623, -0.3641,  0.2798,  ..., -0.5844, -0.3658,  0.5741],
         [ 0.4184, -0.4971, -0.1429,  ...,  0.1260,  0.3042, -0.4440],
         [ 0.0213, -0.0259, -1.1278,  ...,  0.2184, -0.7176, -0.2370]]],
       grad_fn=<AddBackward0>)


# PROTBERT

https://github.com/agemagician/ProtTrans

ProtBert BFD

In [6]:
from transformers import BertForMaskedLM, BertTokenizer, pipeline

tokenizer = BertTokenizer.from_pretrained('../../models/prot_bert_bfd', do_lower_case=False )
model = BertForMaskedLM.from_pretrained("../../models/prot_bert_bfd")
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)
unmasker('D L I P T S S K L V V [MASK] D T S L Q V K K A F F A L V T')

Some weights of the model checkpoint at ../../models/prot_bert_bfd were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.11656127125024796,
  'token': 5,
  'token_str': 'L',
  'sequence': 'D L I P T S S K L V V L D T S L Q V K K A F F A L V T'},
 {'score': 0.08976069837808609,
  'token': 8,
  'token_str': 'V',
  'sequence': 'D L I P T S S K L V V V D T S L Q V K K A F F A L V T'},
 {'score': 0.08864379674196243,
  'token': 10,
  'token_str': 'S',
  'sequence': 'D L I P T S S K L V V S D T S L Q V K K A F F A L V T'},
 {'score': 0.06227641925215721,
  'token': 6,
  'token_str': 'A',
  'sequence': 'D L I P T S S K L V V A D T S L Q V K K A F F A L V T'},
 {'score': 0.06194964423775673,
  'token': 15,
  'token_str': 'T',
  'sequence': 'D L I P T S S K L V V T D T S L Q V K K A F F A L V T'}]

Obtener features de una proteina

In [10]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained('../../models/prot_bert_bfd', do_lower_case=False )
model = BertModel.from_pretrained("../../models/prot_bert_bfd")
sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)

print(output)

Some weights of the model checkpoint at ../../models//prot_bert_bfd were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0787,  0.0253,  0.1002,  ..., -0.1192, -0.0856,  0.0158],
         [ 0.0555, -0.1046, -0.0325,  ...,  0.0509,  0.0432,  0.1018],
         [ 0.1390, -0.0466,  0.0219,  ...,  0.0694,  0.1476,  0.0650],
         ...,
         [ 0.0813, -0.1093, -0.0302,  ...,  0.0872,  0.0206,  0.0516],
         [ 0.0620, -0.0642, -0.0204,  ..., -0.0280,  0.0884,  0.0753],
         [ 0.0979, -0.0240, -0.0724,  ..., -0.0230,  0.0364,  0.0508]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.3145, -0.2646,  0.3113,  ..., -0.3111, -0.3176,  0.3123]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)


ProtBert ProtT5-XL-UniRef50

In [6]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('../../models/prot_t5_xl_half_uniref50-enc', do_lower_case=False).to(device)

# Load the model
model = T5EncoderModel.from_pretrained("../../models/prot_t5_xl_half_uniref50-enc").to(device)

# only GPUs support half-precision currently; if you want to run on CPU use full-precision (not recommended, much slower)
model.full() if device=='cpu' else model.half()

# prepare your protein sequences as a list
sequence_examples = ["PRTEINO", "SEQWENCE"]

# replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]

# tokenize sequences and pad up to the longest sequence in the batch
ids = tokenizer.batch_encode_plus(sequences_example, add_special_tokens=True, padding="longest")

input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

# generate embeddings
with torch.no_grad():
    embedding_rpr = model(input_ids=input_ids,attention_mask=attention_mask)

# extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7]) 
emb_0 = embedding_repr.last_hidden_state[0,:7] # shape (7 x 1024)
# same for the second ([1,:]) sequence but taking into account different sequence lengths ([1,:8])
emb_1 = embedding_repr.last_hidden_state[1,:8] # shape (8 x 1024)

# if you want to derive a single representation (per-protein embedding) for the whole protein
emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)

TypeError: 'NoneType' object is not callable